## 常见的优化有以下几个方面：
* SQL及索引
* 数据库表结构
* 系统配置
* 硬件
<br/>
上面这些越来越成本高，但是优化效果越来越差。

### 如何发现有问题的SQL？
使用MYSQL慢查询日志对有效率问题的SQL进行监控：
* `show variables like 'slow_query_log'`
* `set global slow_query_log_file='/home/mysql/sql_log/mysql-slow.log'`这个是指定了慢查询的日志的位置在哪里。
* `set global log_queries_not_using_indexs=on;`指定是否把没有使用索引的SQL记录到慢查询日志中，通常是需要的。
* `set global long_query_time=1`指定把大于多少秒的SQL记录到慢查询日志中。

#### 慢查询日志包含的内容
* 执行SQL的主机信息:# User@Host:root[root]@localhost[]
* SQL的执行信息: # Query_time:0.00024 Lock_time:0.000000 Rows_sent:0 Rows_examined:0
* SQL执行时间: SET timestamp=1408275856;
* SQL的内容: select CONCAT();

#### 慢查日志的分析工具
* mysqldumpslow(MySQL官方的，会随着MySQL自动安装到我们的服务器中， -h来显示说明)
* pt-query-digest(慢查日志的分析工具)
    * 输出到文件: pt-query-digest slow.log > slow_log.report
    * 输出到数据库表: pt-query-digest slow.log -review h=127.0.0.1,D=test,p=root,P=3306,u=root,t=query_review --create-reviewtable --review-hostory t=hostname_slow。其中的`review`参数是把结果放单数据库的一个表里。通过`create-reviewtable`来建立相关的存储表。
    * 输出的内容包含了三个部分：
        1. 头信息，显示出了日志的时间范围，以及总的SQL数量和不同的SQL数量。
        2. `ResponseTime`:响应时间占比，`Calls`:sql执行次数。
        3. 具体的SQL是什么。

#### 如何通过慢查日志发现有问题的SQL。
1. **查询次数多且每次查询占用时间长的SQL。**通常为pt-query-digest分析的前几个查询。
2. **IO大的SQL。**注意pt-query-digest分析中的`Rows examine`项。
3. **未命中索引的SQL。**注意pt-query-digest分析中`Rows examine`(表示扫描的行数)和`Rows Send`(表示发送的行数)的对比。

#### 通过explain查询和分析SQL的执行计划
通过explain `[我们自己的SQL]`来看执行的计划。
`explain`返回各列的含义：
* `table`:显示这一行的数据是关于哪张表的；
* `type`:这是重要的列，显示连接使用了何种类型。**从最好到最差的连接类型为`const`(表示是长度查找，一般是主键查找)、`eq_reg`(范围查找，一般是唯一索引和主键的 范围查找)、`ref`(常见于连接的查询中，基于某一个索引查找)、`range`(基于索引的范围查找)、`index`(通常是对于索引的扫描)和`ALL`(实际上是进行表扫描)。**
* `possible_keys`:显示可能应用在这张表中的索引。如果为空，没有可能的索引。
* `key`: 实际使用的索引。若果为`NULL`，则没有使用索引。
* `key_len`: 使用的索引的长度。在不损失精确性的情况下，长度越短越好。
* `ref`: 显示索引的哪一列被使用了，如果可能的话，是一个常数。
* `rows`: MySQL认为必须检查的用来返回请求数据的行数。
* `extra`：列需要注意的返回值：
    * `Using filesort`：看到这个的时候，查询就需要优化了。MYSQL需要进行额外的步骤来发现如何对返回的行排序。它根据连接类型以及存储排序键值和匹配条件的全部行的指针来排序全部行。
    * `Using temporary`：看到这个的时候，查询需要优化了。这里，MYSQL需要创建一个临时表来存储结果，这通常发生在对不同的列集进行`ORDER BY`上，而不是`GROUP BY`上。